# Accelerating System Adequacy Assessment using the Multilevel Monte Carlo Approach
Author: Simon Tindemans, s.h.tindemans@tudelft.nl, Delft University of Technology, 2019-2020.
Released under the MIT license.

This notebook generates results that are similar\* to those in the paper *Accelerating System Adequacy Assessment using the Multilevel Monte Carlo Approach* by Simon Tindemans and Goran Strbac, to be presented at the PSCC 2020 conference. 

A preprint of the paper is available here: https://arxiv.org/abs/1910.13013. If you use (portions of) this code, please reference the published paper.

\*: Similarity only due to dependence on execution time

In [1]:
import numpy as np
import logging
import pandas as pd

In [2]:
import MCCoordinator
import HL12Sampler
import StorageSampler

In [3]:
# initialise logging facility
log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
    # initialise global logging facility

logging.basicConfig(level=logging.INFO)


# RTS Case study

In [4]:
def load_grid(base_name, **kwargs):
    """Instantiates an RTS system, after loading parameters from file."""
    line_filename = "%s-lines.txt" % base_name
    line_data = np.genfromtxt(line_filename, dtype=(int, int, float, float, float, float))

    node_filename = "%s-nodes.txt" % base_name
    node_data = np.genfromtxt(node_filename, dtype=float)

    generator_filename = "%s-generators.txt" % base_name
    generator_data = np.genfromtxt(generator_filename, dtype=(float, int, float))

    load_profile_filename = "%s-load-profile.txt" % base_name
    load_profile_data = np.loadtxt(load_profile_filename)
    load_profile_data *= 1.0 / load_profile_data.max()

    return HL12Sampler.HLSystem(node_data, line_data, generator_data, load_profile_data, **kwargs)

In [5]:
RTS_system = load_grid('../data/RTS/RTS', line_capacity_scaling=0.8, line_for_scaling=1)

## Reference: simple MC

Used in Table I, Table II, Table IV

In [6]:
mcc = MCCoordinator.MCCoordinator(RTS_system, ml_hierarchy=['HL2full'], use_expectations=True, use_joblib=False, joblib_batch_size=100)
mcc.explore(n_samples=100)
for i in range(10):
    mcc.run_recommended(time_seconds=60, optimization_target='EPNS', verbose=False)

Running: {frozenset({'HL2full'}): 100}
relative sample numbers:  {frozenset({'HL2full'}): 1.0}
Running: {frozenset({'HL2full'}): 7990}
relative sample numbers:  {frozenset({'HL2full'}): 121.81010572188843}
Running: {frozenset({'HL2full'}): 10868}
relative sample numbers:  {frozenset({'HL2full'}): 130.4519106991025}
Running: {frozenset({'HL2full'}): 10713}
relative sample numbers:  {frozenset({'HL2full'}): 134.39131159262914}
Running: {frozenset({'HL2full'}): 10890}
relative sample numbers:  {frozenset({'HL2full'}): 128.35238247963477}
Running: {frozenset({'HL2full'}): 10961}
relative sample numbers:  {frozenset({'HL2full'}): 118.95116216746976}
Running: {frozenset({'HL2full'}): 10978}
relative sample numbers:  {frozenset({'HL2full'}): 114.63099120263082}
Running: {frozenset({'HL2full'}): 10997}
relative sample numbers:  {frozenset({'HL2full'}): 109.29699700345202}
Running: {frozenset({'HL2full'}): 11017}
relative sample numbers:  {frozenset({'HL2full'}): 108.3405552565517}
Running: {fr

In [7]:
mcc.verbose_result()


Wall clock time: 582.0 s 	Total sample generation: 580.0 s 	Index specialisation: 577.0 s
LOLP : 0.001709  +- 0.00013  	computational speed:  0.313 	estimated speedup:  0.998
EPNS : 0.2378 MW +- 0.024 MW 	computational speed:  0.173 	estimated speedup:  0.998

Index set and time estimates:
HL2full : 0.0054 s


RESULTS BY ML INDEX
index: HL2full 	count:  106510 	avg specialisation time: 5.42 ms
	 LOLP : 0.001709  	+- 0.00013 
	 EPNS : 0.2378 MW 	+- 0.024 MW

RESULTS BY SET
{'HL2full'} 	count:  106510 	avg exec time:  5.45 ms

RESULTS BY TELESCOPIC TERM
{'HL2full'}
	 LOLP contribution: 0.001709  	+- 0.00013  	sample correlation: n/a
	 EPNS contribution: 0.2378 MW 	+- 0.024 MW 	sample correlation: n/a


# HL2 full

Used in Table II, Table III, Table IV

In [8]:
mcc = MCCoordinator.MCCoordinator(RTS_system, ml_hierarchy=['HL2full', 'HL1'], use_expectations=True, use_joblib=False, joblib_batch_size=100)
mcc.explore(n_samples=100)
for i in range(10):
    mcc.run_recommended(time_seconds=60, optimization_target='EPNS', verbose=False)

Running: {frozenset({'HL1', 'HL2full'}): 100, frozenset({'HL1'}): 100}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 1.0, frozenset({'HL1'}): 1.0}
Running: {frozenset({'HL1', 'HL2full'}): 10962, frozenset({'HL1'}): 10962}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 32.46702973057098, frozenset({'HL1'}): 0.0}
Running: {frozenset({'HL1', 'HL2full'}): 11162, frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 26.907360009664206, frozenset({'HL1'}): 0.0}
Running: {frozenset({'HL1', 'HL2full'}): 11097, frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 25.724508257700947, frozenset({'HL1'}): 0.0}
Running: {frozenset({'HL1', 'HL2full'}): 11124, frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 26.396291348885942, frozenset({'HL1'}): 0.0}
Running: {frozenset({'HL1', 'HL2full'}): 11115, frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 26.0614583727

In [9]:
mcc.verbose_result()


Wall clock time: 601.0 s 	Total sample generation: 599.0 s 	Index specialisation: 596.0 s
LOLP : 0.001457  +- 5.8e-05  	computational speed:  1.04 	estimated speedup:  3.7
EPNS : 0.1863 MW +- 0.0048 MW 	computational speed:  2.54 	estimated speedup:  18.1

Index set and time estimates:
HL2full : 0.0054 s
HL1 : 0.0 s


RESULTS BY ML INDEX
index: HL1 	count:  111184 	avg specialisation time: 0.0229 ms
	 LOLP : 0.001079  	+- 0.0 
	 EPNS : 0.1346 MW 	+- 0.0 MW
index: HL2full 	count:  111184 	avg specialisation time: 5.34 ms
	 LOLP : 0.001403  	+- 0.00011 
	 EPNS : 0.1867 MW 	+- 0.02 MW

RESULTS BY SET
{'HL1', 'HL2full'} 	count:  111184 	avg exec time:  5.39 ms
{'HL1'} 	count:  0 	avg exec time:  0.0 ms

RESULTS BY TELESCOPIC TERM
{'HL1', 'HL2full'}
	 LOLP contribution: 0.0003778  	+- 5.8e-05  	sample correlation: 0.855
	 EPNS contribution: 0.05165 MW 	+- 0.0048 MW 	sample correlation: 0.983
{'HL1'}
	 LOLP contribution: 0.001079  	+- 0.0  	sample correlation: n/a
	 EPNS contribution: 0.134

# HL2 full - no expectation
Used in Table II

In [10]:
mcc = MCCoordinator.MCCoordinator(RTS_system, ml_hierarchy=['HL2full', 'HL1'], use_expectations=False, use_joblib=False, joblib_batch_size=100)
mcc.explore(n_samples=1000)
for i in range(10):
    mcc.run_recommended(time_seconds=60, optimization_target='EPNS', verbose=False)

Running: {frozenset({'HL1', 'HL2full'}): 1000, frozenset({'HL1'}): 1000}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 86.56682339482288, frozenset({'HL1'}): 3581.698353524935}
Running: {frozenset({'HL1', 'HL2full'}): 8974, frozenset({'HL1'}): 371337}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 36.436918237415796, frozenset({'HL1'}): 1756.0837624376825}
Running: {frozenset({'HL1', 'HL2full'}): 9229, frozenset({'HL1'}): 444828}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 29.738449107286193, frozenset({'HL1'}): 1434.6360384774873}
Running: {frozenset({'HL1', 'HL2full'}): 9247, frozenset({'HL1'}): 446098}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 29.424799150312076, frozenset({'HL1'}): 1418.5196394030932}
Running: {frozenset({'HL1', 'HL2full'}): 9236, frozenset({'HL1'}): 445269}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 30.651615173662208, frozenset({'HL1'}): 1477.1587396556608}
Running: {frozenset({'HL1', 'HL2full'})

In [11]:
mcc.verbose_result()


Wall clock time: 627.0 s 	Total sample generation: 605.0 s 	Index specialisation: 554.0 s
LOLP : 0.001498  +- 6.7e-05  	computational speed:  0.793 	estimated speedup:  2.72
EPNS : 0.1898 MW +- 0.0058 MW 	computational speed:  1.73 	estimated speedup:  10.3

Index set and time estimates:
HL2full : 0.0054 s
HL1 : 2.3e-05 s


RESULTS BY ML INDEX
index: HL1 	count:  4473352 	avg specialisation time: 0.0123 ms
	 LOLP : 0.0011  	+- 1.6e-05 
	 EPNS : 0.139 MW 	+- 0.0027 MW
index: HL2full 	count:  93158 	avg specialisation time: 5.36 ms
	 LOLP : 0.001428  	+- 0.00012 
	 EPNS : 0.1821 MW 	+- 0.021 MW

RESULTS BY SET
{'HL1', 'HL2full'} 	count:  93158 	avg exec time:  5.41 ms
{'HL1'} 	count:  4380194 	avg exec time:  0.0232 ms

RESULTS BY TELESCOPIC TERM
{'HL1', 'HL2full'}
	 LOLP contribution: 0.0003972  	+- 6.5e-05  	sample correlation: 0.849
	 EPNS contribution: 0.05066 MW 	+- 0.0051 MW 	sample correlation: 0.98
{'HL1'}
	 LOLP contribution: 0.001101  	+- 1.6e-05  	sample correlation: n/a
	 EP

## RTS - line scale 0.9

Comparison of MLMC with conventional MC - Table IV

In [12]:
RTS_system = load_grid('../data/RTS/RTS', line_capacity_scaling=0.9, line_for_scaling=1)

In [13]:
mcc = MCCoordinator.MCCoordinator(RTS_system, ml_hierarchy=['HL2full', 'HL1'], use_expectations=True, use_joblib=False, joblib_batch_size=100)
mcc.explore(n_samples=100)
for i in range(10):
    mcc.run_recommended(time_seconds=60, optimization_target='EPNS', verbose=False)
mcc.verbose_result()

Running: {frozenset({'HL1', 'HL2full'}): 100, frozenset({'HL1'}): 100}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 1.0, frozenset({'HL1'}): 1.0}
Running: {frozenset({'HL1', 'HL2full'}): 11259, frozenset({'HL1'}): 11259}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 29.399122516760194, frozenset({'HL1'}): 0.0}
Running: {frozenset({'HL1', 'HL2full'}): 11210, frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 25.035392019464723, frozenset({'HL1'}): 0.0}
Running: {frozenset({'HL1', 'HL2full'}): 11176, frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 26.55099065709211, frozenset({'HL1'}): 0.0}
Running: {frozenset({'HL1', 'HL2full'}): 11178, frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 25.642775753065777, frozenset({'HL1'}): 0.0}
Running: {frozenset({'HL1', 'HL2full'}): 11174, frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 25.9017476191

In [14]:

mcc = MCCoordinator.MCCoordinator(RTS_system, ml_hierarchy=['HL2full'], use_expectations=True, use_joblib=False, joblib_batch_size=100)
mcc.explore(n_samples=100)
for i in range(10):
    mcc.run_recommended(time_seconds=60, optimization_target='EPNS', verbose=False)
mcc.verbose_result()

Running: {frozenset({'HL2full'}): 100}
relative sample numbers:  {frozenset({'HL2full'}): 1.0}
Running: {frozenset({'HL2full'}): 11144}
relative sample numbers:  {frozenset({'HL2full'}): 108.8044563753181}
Running: {frozenset({'HL2full'}): 11217}
relative sample numbers:  {frozenset({'HL2full'}): 100.08177135831447}
Running: {frozenset({'HL2full'}): 11218}
relative sample numbers:  {frozenset({'HL2full'}): 90.0817001769497}
Running: {frozenset({'HL2full'}): 11224}
relative sample numbers:  {frozenset({'HL2full'}): 86.78354026378999}
Running: {frozenset({'HL2full'}): 11220}
relative sample numbers:  {frozenset({'HL2full'}): 91.53300166894468}
Running: {frozenset({'HL2full'}): 11222}
relative sample numbers:  {frozenset({'HL2full'}): 90.81050963931158}
Running: {frozenset({'HL2full'}): 11230}
relative sample numbers:  {frozenset({'HL2full'}): 93.06526227328709}
Running: {frozenset({'HL2full'}): 11231}
relative sample numbers:  {frozenset({'HL2full'}): 94.42580994210196}
Running: {frozens

## RTS - line scale 1.0

Comparison between MLMC and regular MC - used in Table IV

In [15]:
RTS_system = load_grid('../data/RTS/RTS', line_capacity_scaling=1.0, line_for_scaling=1)

In [16]:
mcc = MCCoordinator.MCCoordinator(RTS_system, ml_hierarchy=['HL2full', 'HL1'], use_expectations=True, use_joblib=False, joblib_batch_size=100)
mcc.explore(n_samples=100)
for i in range(10):
    mcc.run_recommended(time_seconds=60, optimization_target='EPNS', verbose=False)
mcc.verbose_result()

Running: {frozenset({'HL1', 'HL2full'}): 100, frozenset({'HL1'}): 100}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 1.0, frozenset({'HL1'}): 1.0}
Running: {frozenset({'HL1', 'HL2full'}): 11628, frozenset({'HL1'}): 11628}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 23.19421418611651, frozenset({'HL1'}): 0.0}
Running: {frozenset({'HL1', 'HL2full'}): 11356, frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 18.96460348256543, frozenset({'HL1'}): 0.0}
Running: {frozenset({'HL1', 'HL2full'}): 11385, frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 16.511310925437236, frozenset({'HL1'}): 0.0}
Running: {frozenset({'HL1', 'HL2full'}): 11391, frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 19.582530573077868, frozenset({'HL1'}): 0.0}
Running: {frozenset({'HL1', 'HL2full'}): 11396, frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1', 'HL2full'}): 18.14868437110

In [17]:
mcc = MCCoordinator.MCCoordinator(RTS_system, ml_hierarchy=['HL2full'], use_expectations=True, use_joblib=False, joblib_batch_size=100)
mcc.explore(n_samples=100)
for i in range(10):
    mcc.run_recommended(time_seconds=60, optimization_target='EPNS', verbose=False)
mcc.verbose_result()

Running: {frozenset({'HL2full'}): 100}
relative sample numbers:  {frozenset({'HL2full'}): 1.0}
Running: {frozenset({'HL2full'}): 11023}
relative sample numbers:  {frozenset({'HL2full'}): 56.500739870717375}
Running: {frozenset({'HL2full'}): 11494}
relative sample numbers:  {frozenset({'HL2full'}): 72.37066084914309}
Running: {frozenset({'HL2full'}): 11465}
relative sample numbers:  {frozenset({'HL2full'}): 78.79767987347054}
Running: {frozenset({'HL2full'}): 11495}
relative sample numbers:  {frozenset({'HL2full'}): 86.94747356578667}
Running: {frozenset({'HL2full'}): 11496}
relative sample numbers:  {frozenset({'HL2full'}): 93.27942540373935}
Running: {frozenset({'HL2full'}): 11470}
relative sample numbers:  {frozenset({'HL2full'}): 90.63565421627727}
Running: {frozenset({'HL2full'}): 11474}
relative sample numbers:  {frozenset({'HL2full'}): 92.3509282713587}
Running: {frozenset({'HL2full'}): 11462}
relative sample numbers:  {frozenset({'HL2full'}): 87.93832283100912}
Running: {frozens

## Simple HL1 run
Used in Table I

In [8]:
RTS_system = load_grid('../data/RTS/RTS', line_capacity_scaling=0.8, line_for_scaling=1)

In [10]:
mcc = MCCoordinator.MCCoordinator(RTS_system, ml_hierarchy=['HL1'], use_expectations=True, use_joblib=False, joblib_batch_size=100)
mcc.explore(n_samples=100)
for i in range(10):
    mcc.run_recommended(time_seconds=60, optimization_target='EPNS', verbose=False)
mcc.verbose_result()

Running: {frozenset({'HL1'}): 100}
relative sample numbers:  {frozenset({'HL1'}): 1.0}
Running: {frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1'}): 1.0}
Running: {frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1'}): 1.0}
Running: {frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1'}): 1.0}
Running: {frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1'}): 1.0}
Running: {frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1'}): 1.0}
Running: {frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1'}): 1.0}
Running: {frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1'}): 1.0}
Running: {frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1'}): 1.0}
Running: {frozenset({'HL1'}): 1}
relative sample numbers:  {frozenset({'HL1'}): 1.0}
Running: {frozenset({'HL1'}): 1}

Wall clock time: 0.04 s 	Total sample generation: 0.0 s 	Index specialisation: 0.0 s
LOLP : 0.001079  +- 0.0  	com

/Users/stindemans/Dropbox/code/MLMC/code/SummaryStats.py:83: RuntimeWarning: invalid value encountered in multiply
  return self.m2/(self.count - 1) if (self.count > 1) else (math.inf * self.m2)


In [11]:
mcc = MCCoordinator.MCCoordinator(RTS_system, ml_hierarchy=['HL1'], use_expectations=False, use_joblib=False, joblib_batch_size=100)
mcc.explore(n_samples=100)
for i in range(10):
    mcc.run_recommended(time_seconds=60, optimization_target='EPNS', verbose=False)
mcc.verbose_result()

Running: {frozenset({'HL1'}): 100}
relative sample numbers:  {frozenset({'HL1'}): 1.0}
Running: {frozenset({'HL1'}): 1391761}
relative sample numbers:  {frozenset({'HL1'}): 1058.5073617067874}
Running: {frozenset({'HL1'}): 2209667}
relative sample numbers:  {frozenset({'HL1'}): 1066.4777233367859}
Running: {frozenset({'HL1'}): 2243240}
relative sample numbers:  {frozenset({'HL1'}): 1083.1642776383667}
Running: {frozenset({'HL1'}): 2253104}
relative sample numbers:  {frozenset({'HL1'}): 1070.697177813339}
Running: {frozenset({'HL1'}): 2260029}
relative sample numbers:  {frozenset({'HL1'}): 1073.6140512683685}
Running: {frozenset({'HL1'}): 2263048}
relative sample numbers:  {frozenset({'HL1'}): 1076.9567545575485}
Running: {frozenset({'HL1'}): 2265000}
relative sample numbers:  {frozenset({'HL1'}): 1076.9317877344613}
Running: {frozenset({'HL1'}): 2263658}
relative sample numbers:  {frozenset({'HL1'}): 1078.8413266159812}
Running: {frozenset({'HL1'}): 2266688}
relative sample numbers:  {

# Storage case study

In [14]:
def storage_system(**kwargs):
    """
    Instantiate StorageSystem object using GB system data

    :param wind_power: assumed wind power capacity (in MW)
    :param kwargs: additional arguments to be supplied to the StorageSystem constructor
    :return: StorageSystem object
    """

    data = pd.read_csv('../data/UKdata/20161213_uk_wind_solar_demand_temperature.csv',
                       parse_dates=['UTC Time', 'Local Time'], infer_datetime_format=True, dayfirst=True, index_col=0)

    demand_data = data['demand_net'].dropna()['2006':'2015']
    wind_data = 10000 * data['wind_merra1'].dropna()

    demand_samples = {yeardata[0]: yeardata[1].values[:8760] for yeardata in
                      demand_data.groupby(demand_data.index.year)}
    wind_samples = {yeardata[0]: yeardata[1].values[:8760] for yeardata in wind_data.groupby(wind_data.index.year)}

    dataframe = pd.read_csv('../data/UKdata/battery_data.csv')
    store_power_list=3*dataframe['Power (MW)'][0:27]
    store_energy_list=3*dataframe['Energy (MWh)'][0:27]

    return StorageSampler.StorageSystem(demand_samples=demand_samples, 
                                        wind_samples=wind_samples, 
                                        store_power_list=store_power_list,
                                        store_energy_list=store_energy_list,
                                        **kwargs)

In [15]:
# define and initialise system
system = storage_system()

Generator set: [1200, 600, 600, 250, 250, 120, 60, 20, 20, 10, 10, 10]
Optimising number of generator sets to LOLE target of 3 hours
20 generator sets. Base LOLE=2.92 h; determined load offset of 36.15 MW to reach LOLE target.



## Optimal combination: M2, M1, M0
Used in Table VI, VII

In [20]:
# set up MLMC coordinator with link to system
mcc = MCCoordinator.MCCoordinator(factory=system, 
                                  ml_hierarchy=['OptimalNStore','GreedyNStore','AvgStore'], 
                                  use_expectations=True, 
                                  use_joblib=False, joblib_n_jobs=-1, joblib_batch_size=5)

mcc.explore(n_samples=20)

for i in range(10):
    mcc.run_recommended(time_seconds=60, verbose=True, optimization_target='EENS')


Running: {frozenset({'OptimalNStore', 'GreedyNStore'}): 20, frozenset({'GreedyNStore', 'AvgStore'}): 20, frozenset({'AvgStore'}): 20}
Optimizing for EENS
relative sample numbers:  {frozenset({'OptimalNStore', 'GreedyNStore'}): 838.4662620071337, frozenset({'GreedyNStore', 'AvgStore'}): 7577.535052427264, frozenset({'AvgStore'}): 0.0}
Running: {frozenset({'OptimalNStore', 'GreedyNStore'}): 17, frozenset({'GreedyNStore', 'AvgStore'}): 154, frozenset({'AvgStore'}): 1}
Optimizing for EENS
relative sample numbers:  {frozenset({'OptimalNStore', 'GreedyNStore'}): 640.2329329488565, frozenset({'GreedyNStore', 'AvgStore'}): 6342.134667573733, frozenset({'AvgStore'}): 0.0}
Running: {frozenset({'OptimalNStore', 'GreedyNStore'}): 17, frozenset({'GreedyNStore', 'AvgStore'}): 177, frozenset({'AvgStore'}): 1}
Optimizing for EENS
relative sample numbers:  {frozenset({'OptimalNStore', 'GreedyNStore'}): 597.56062366947, frozenset({'GreedyNStore', 'AvgStore'}): 5987.682376047066, frozenset({'AvgStore'}):

In [21]:

mcc.verbose_result()


Wall clock time: 615.0 s 	Total sample generation: 614.0 s 	Index specialisation: 606.0 s
LOLE : 1.722 h +- 0.026 h 	computational speed:  6.88 	estimated speedup:  52.6
EENS : 2397.0 MWh +- 9.1 MWh 	computational speed:  112.0 	estimated speedup:  967.0

Index set and time estimates:
OptimalNStore : 1.7 s
GreedyNStore : 0.17 s
AvgStore : 0.0 s


RESULTS BY ML INDEX
index: OptimalNStore 	count:  190 	avg specialisation time: 1500.0 ms
	 LOLE : 1.584 h 	+- 0.27 h
	 EENS : 1862.0 MWh 	+- 400.0 MWh
index: GreedyNStore 	count:  1961 	avg specialisation time: 164.0 ms
	 LOLE : 1.787 h 	+- 0.099 h
	 EENS : 2436.0 MWh 	+- 180.0 MWh
index: AvgStore 	count:  1771 	avg specialisation time: 0.0425 ms
	 LOLE : 2.142 h 	+- 0.0 h
	 EENS : 2548.0 MWh 	+- 0.0 MWh

RESULTS BY SET
{'OptimalNStore', 'GreedyNStore'} 	count:  190 	avg exec time:  1670.0 ms
{'GreedyNStore', 'AvgStore'} 	count:  1771 	avg exec time:  167.0 ms
{'AvgStore'} 	count:  0 	avg exec time:  0.0 ms

RESULTS BY TELESCOPIC TERM
{'Opti

## Comparison: two level M2, M0
Used in Table VI

In [22]:
# set up MLMC coordinator with link to system
mcc = MCCoordinator.MCCoordinator(factory=system, 
                                  ml_hierarchy=['OptimalNStore', 'AvgStore'], 
                                  use_expectations=True, 
                                  use_joblib=False, joblib_n_jobs=-1, joblib_batch_size=5)

mcc.explore(n_samples=20)
for i in range(10):
    mcc.run_recommended(time_seconds=60, optimization_target='EENS')

Running: {frozenset({'OptimalNStore', 'AvgStore'}): 20, frozenset({'AvgStore'}): 20}
relative sample numbers:  {frozenset({'OptimalNStore', 'AvgStore'}): 2968.7847403671776, frozenset({'AvgStore'}): 0.0}
Running: {frozenset({'OptimalNStore', 'AvgStore'}): 39, frozenset({'AvgStore'}): 1}
relative sample numbers:  {frozenset({'OptimalNStore', 'AvgStore'}): 2322.468854882883, frozenset({'AvgStore'}): 0.0}
Running: {frozenset({'OptimalNStore', 'AvgStore'}): 39, frozenset({'AvgStore'}): 1}
relative sample numbers:  {frozenset({'OptimalNStore', 'AvgStore'}): 1924.7343592590755, frozenset({'AvgStore'}): 0.0}
Running: {frozenset({'OptimalNStore', 'AvgStore'}): 39, frozenset({'AvgStore'}): 1}
relative sample numbers:  {frozenset({'OptimalNStore', 'AvgStore'}): 1674.716530574329, frozenset({'AvgStore'}): 0.0}
Running: {frozenset({'OptimalNStore', 'AvgStore'}): 39, frozenset({'AvgStore'}): 1}
relative sample numbers:  {frozenset({'OptimalNStore', 'AvgStore'}): 1671.256426649425, frozenset({'AvgSt

In [23]:
mcc.verbose_result()


Wall clock time: 618.0 s 	Total sample generation: 618.0 s 	Index specialisation: 617.0 s
LOLE : 1.747 h +- 0.051 h 	computational speed:  1.88 	estimated speedup:  19.4
EENS : 2415.0 MWh +- 16.0 MWh 	computational speed:  38.1 	estimated speedup:  797.0

Index set and time estimates:
OptimalNStore : 1.5 s
AvgStore : 0.0 s


RESULTS BY ML INDEX
index: OptimalNStore 	count:  410 	avg specialisation time: 1500.0 ms
	 LOLE : 1.79 h 	+- 0.23 h
	 EENS : 2408.0 MWh 	+- 440.0 MWh
index: AvgStore 	count:  410 	avg specialisation time: 0.0558 ms
	 LOLE : 2.142 h 	+- 0.0 h
	 EENS : 2548.0 MWh 	+- 0.0 MWh

RESULTS BY SET
{'OptimalNStore', 'AvgStore'} 	count:  410 	avg exec time:  1510.0 ms
{'AvgStore'} 	count:  0 	avg exec time:  0.0 ms

RESULTS BY TELESCOPIC TERM
{'OptimalNStore', 'AvgStore'}
	 LOLE contribution: -0.3951 h 	+- 0.051 h 	sample correlation: 0.978
	 EENS contribution: -132.8 MWh 	+- 16.0 MWh 	sample correlation: 0.99
{'AvgStore'}
	 LOLE contribution: 2.142 h 	+- 0.0 h 	sample corr

## Comparison: direct simulation
Used in Table V and VI

In [24]:
# set up MLMC coordinator with link to system
mcc = MCCoordinator.MCCoordinator(factory=system, 
                                  ml_hierarchy=['OptimalNStore'], 
                                  use_expectations=True,
                                  use_joblib=False, joblib_n_jobs=-1, joblib_batch_size=5)

mcc.explore(n_samples=20)
for i in range(10):
    mcc.run_recommended(time_seconds=60, optimization_target='EENS')

Running: {frozenset({'OptimalNStore'}): 20}
relative sample numbers:  {frozenset({'OptimalNStore'}): 6082.986766288706}
Running: {frozenset({'OptimalNStore'}): 39}
relative sample numbers:  {frozenset({'OptimalNStore'}): 10815.754174148093}
Running: {frozenset({'OptimalNStore'}): 39}
relative sample numbers:  {frozenset({'OptimalNStore'}): 8594.659304990557}
Running: {frozenset({'OptimalNStore'}): 39}
relative sample numbers:  {frozenset({'OptimalNStore'}): 7355.328352647725}
Running: {frozenset({'OptimalNStore'}): 39}
relative sample numbers:  {frozenset({'OptimalNStore'}): 6649.740412700156}
Running: {frozenset({'OptimalNStore'}): 39}
relative sample numbers:  {frozenset({'OptimalNStore'}): 6201.872283158146}
Running: {frozenset({'OptimalNStore'}): 39}
relative sample numbers:  {frozenset({'OptimalNStore'}): 6019.982603699087}
Running: {frozenset({'OptimalNStore'}): 39}
relative sample numbers:  {frozenset({'OptimalNStore'}): 6101.86323528039}
Running: {frozenset({'OptimalNStore'}): 

In [25]:
mcc.verbose_result()


Wall clock time: 620.0 s 	Total sample generation: 620.0 s 	Index specialisation: 619.0 s
LOLE : 1.539 h +- 0.19 h 	computational speed:  0.105 	estimated speedup:  1.0
EENS : 2135.0 MWh +- 370.0 MWh 	computational speed:  0.0533 	estimated speedup:  1.0

Index set and time estimates:
OptimalNStore : 1.5 s


RESULTS BY ML INDEX
index: OptimalNStore 	count:  410 	avg specialisation time: 1510.0 ms
	 LOLE : 1.539 h 	+- 0.19 h
	 EENS : 2135.0 MWh 	+- 370.0 MWh

RESULTS BY SET
{'OptimalNStore'} 	count:  410 	avg exec time:  1510.0 ms

RESULTS BY TELESCOPIC TERM
{'OptimalNStore'}
	 LOLE contribution: 1.539 h 	+- 0.19 h 	sample correlation: n/a
	 EENS contribution: 2135.0 MWh 	+- 370.0 MWh 	sample correlation: n/a


## Comparison: three-level with no-store (M2-M1-M0')
Used in Table VI

In [26]:
# set up MLMC coordinator with link to system
mcc = MCCoordinator.MCCoordinator(factory=system, 
                                  ml_hierarchy=['OptimalNStore','GreedyNStore','NoStore'], 
                                  use_expectations=True, 
                                  use_joblib=False, joblib_n_jobs=-1, joblib_batch_size=5)

mcc.explore(n_samples=20)
for i in range(10):
    mcc.run_recommended(time_seconds=60, optimization_target='EENS')

Running: {frozenset({'OptimalNStore', 'GreedyNStore'}): 20, frozenset({'GreedyNStore', 'NoStore'}): 20, frozenset({'NoStore'}): 20}
relative sample numbers:  {frozenset({'OptimalNStore', 'GreedyNStore'}): 474.7771250973566, frozenset({'GreedyNStore', 'NoStore'}): 4842.206523644391, frozenset({'NoStore'}): 0.0}
Running: {frozenset({'OptimalNStore', 'GreedyNStore'}): 17, frozenset({'GreedyNStore', 'NoStore'}): 180, frozenset({'NoStore'}): 1}
relative sample numbers:  {frozenset({'OptimalNStore', 'GreedyNStore'}): 521.0746590484653, frozenset({'GreedyNStore', 'NoStore'}): 5979.708148513961, frozenset({'NoStore'}): 0.0}
Running: {frozenset({'OptimalNStore', 'GreedyNStore'}): 17, frozenset({'GreedyNStore', 'NoStore'}): 196, frozenset({'NoStore'}): 1}
relative sample numbers:  {frozenset({'OptimalNStore', 'GreedyNStore'}): 897.6878774200457, frozenset({'GreedyNStore', 'NoStore'}): 8928.097346575612, frozenset({'NoStore'}): 0.0}
Running: {frozenset({'OptimalNStore', 'GreedyNStore'}): 18, froz

In [27]:
mcc.verbose_result()


Wall clock time: 636.0 s 	Total sample generation: 635.0 s 	Index specialisation: 628.0 s
LOLE : 1.592 h +- 0.06 h 	computational speed:  1.1 	estimated speedup:  9.33
EENS : 2275.0 MWh +- 71.0 MWh 	computational speed:  1.61 	estimated speedup:  23.1

Index set and time estimates:
OptimalNStore : 1.6 s
GreedyNStore : 0.17 s
NoStore : 0.0 s


RESULTS BY ML INDEX
index: OptimalNStore 	count:  198 	avg specialisation time: 1500.0 ms
	 LOLE : 1.323 h 	+- 0.26 h
	 EENS : 1796.0 MWh 	+- 480.0 MWh
index: GreedyNStore 	count:  2041 	avg specialisation time: 163.0 ms
	 LOLE : 1.853 h 	+- 0.11 h
	 EENS : 2625.0 MWh 	+- 210.0 MWh
index: NoStore 	count:  1843 	avg specialisation time: 0.00158 ms
	 LOLE : 3.0 h 	+- 0.0 h
	 EENS : 3799.0 MWh 	+- 0.0 MWh

RESULTS BY SET
{'OptimalNStore', 'GreedyNStore'} 	count:  198 	avg exec time:  1640.0 ms
{'GreedyNStore', 'NoStore'} 	count:  1843 	avg exec time:  168.0 ms
{'NoStore'} 	count:  0 	avg exec time:  0.0 ms

RESULTS BY TELESCOPIC TERM
{'OptimalNStore

## Comparison: direct simulation of less accurate models
Used for Table V. Where possible (models M0 and M0'), results are computed alongside direct estimation of expectations.

In [16]:
# set up MLMC coordinator with link to system
mcc = MCCoordinator.MCCoordinator(factory=system, 
                                  ml_hierarchy=['GreedyNStore'], 
                                  use_expectations=True,
                                  use_joblib=False, joblib_n_jobs=-1, joblib_batch_size=5)

mcc.explore(n_samples=20)
for i in range(10):
    mcc.run_recommended(time_seconds=60, optimization_target='EENS')

Running: {frozenset({'GreedyNStore'}): 20}
relative sample numbers:  {frozenset({'GreedyNStore'}): 15774.070884025656}
Running: {frozenset({'GreedyNStore'}): 234}
relative sample numbers:  {frozenset({'GreedyNStore'}): 21399.43686819916}
Running: {frozenset({'GreedyNStore'}): 272}
relative sample numbers:  {frozenset({'GreedyNStore'}): 21399.83484314624}
Running: {frozenset({'GreedyNStore'}): 305}
relative sample numbers:  {frozenset({'GreedyNStore'}): 32907.059038808104}
Running: {frozenset({'GreedyNStore'}): 309}
relative sample numbers:  {frozenset({'GreedyNStore'}): 30895.94370522232}
Running: {frozenset({'GreedyNStore'}): 317}
relative sample numbers:  {frozenset({'GreedyNStore'}): 28397.98612017454}
Running: {frozenset({'GreedyNStore'}): 322}
relative sample numbers:  {frozenset({'GreedyNStore'}): 27575.574517213885}
Running: {frozenset({'GreedyNStore'}): 327}
relative sample numbers:  {frozenset({'GreedyNStore'}): 26136.945761097166}
Running: {frozenset({'GreedyNStore'}): 336}
r

In [17]:
mcc.verbose_result()


Wall clock time: 556.0 s 	Total sample generation: 556.0 s 	Index specialisation: 543.0 s
LOLE : 1.806 h +- 0.084 h 	computational speed:  0.825 	estimated speedup:  1.0
EENS : 2592.0 MWh +- 180.0 MWh 	computational speed:  0.38 	estimated speedup:  1.0

Index set and time estimates:
GreedyNStore : 0.18 s


RESULTS BY ML INDEX
index: GreedyNStore 	count:  3108 	avg specialisation time: 175.0 ms
	 LOLE : 1.806 h 	+- 0.084 h
	 EENS : 2592.0 MWh 	+- 180.0 MWh

RESULTS BY SET
{'GreedyNStore'} 	count:  3108 	avg exec time:  179.0 ms

RESULTS BY TELESCOPIC TERM
{'GreedyNStore'}
	 LOLE contribution: 1.806 h 	+- 0.084 h 	sample correlation: n/a
	 EENS contribution: 2592.0 MWh 	+- 180.0 MWh 	sample correlation: n/a


In [18]:
# set up MLMC coordinator with link to system
mcc = MCCoordinator.MCCoordinator(factory=system, 
                                  ml_hierarchy=['AvgStore'], 
                                  use_expectations=False,
                                  use_joblib=False, joblib_n_jobs=-1, joblib_batch_size=5)

mcc.explore(n_samples=20)
for i in range(10):
    mcc.run_recommended(time_seconds=60, optimization_target='EENS')

mcc.verbose_result()

Running: {frozenset({'AvgStore'}): 20}
relative sample numbers:  {frozenset({'AvgStore'}): 31541.628450953045}
Running: {frozenset({'AvgStore'}): 16604}
relative sample numbers:  {frozenset({'AvgStore'}): 141875.59344635985}
Running: {frozenset({'AvgStore'}): 16387}
relative sample numbers:  {frozenset({'AvgStore'}): 142742.2667435453}
Running: {frozenset({'AvgStore'}): 16329}
relative sample numbers:  {frozenset({'AvgStore'}): 141038.36514342038}
Running: {frozenset({'AvgStore'}): 16208}
relative sample numbers:  {frozenset({'AvgStore'}): 143354.52093195997}
Running: {frozenset({'AvgStore'}): 16120}
relative sample numbers:  {frozenset({'AvgStore'}): 142929.09968010103}
Running: {frozenset({'AvgStore'}): 16050}
relative sample numbers:  {frozenset({'AvgStore'}): 144871.6815459249}
Running: {frozenset({'AvgStore'}): 16106}
relative sample numbers:  {frozenset({'AvgStore'}): 145911.59604340518}
Running: {frozenset({'AvgStore'}): 16012}
relative sample numbers:  {frozenset({'AvgStore'}):

In [19]:
# set up MLMC coordinator with link to system
mcc = MCCoordinator.MCCoordinator(factory=system, 
                                  ml_hierarchy=['AvgStore'], 
                                  use_expectations=True,
                                  use_joblib=False, joblib_n_jobs=-1, joblib_batch_size=5)

mcc.explore(n_samples=20)
for i in range(10):
    mcc.run_recommended(time_seconds=60, optimization_target='EENS')
    
mcc.verbose_result()

Running: {frozenset({'AvgStore'}): 20}
relative sample numbers:  {frozenset({'AvgStore'}): 1.0}
Running: {frozenset({'AvgStore'}): 1}
relative sample numbers:  {frozenset({'AvgStore'}): 1.0}
Running: {frozenset({'AvgStore'}): 1}
relative sample numbers:  {frozenset({'AvgStore'}): 1.0}
Running: {frozenset({'AvgStore'}): 1}
relative sample numbers:  {frozenset({'AvgStore'}): 1.0}
Running: {frozenset({'AvgStore'}): 1}
relative sample numbers:  {frozenset({'AvgStore'}): 1.0}
Running: {frozenset({'AvgStore'}): 1}
relative sample numbers:  {frozenset({'AvgStore'}): 1.0}
Running: {frozenset({'AvgStore'}): 1}
relative sample numbers:  {frozenset({'AvgStore'}): 1.0}
Running: {frozenset({'AvgStore'}): 1}
relative sample numbers:  {frozenset({'AvgStore'}): 1.0}
Running: {frozenset({'AvgStore'}): 1}
relative sample numbers:  {frozenset({'AvgStore'}): 1.0}
Running: {frozenset({'AvgStore'}): 1}
relative sample numbers:  {frozenset({'AvgStore'}): 1.0}
Running: {frozenset({'AvgStore'}): 1}

Wall clock

/Users/stindemans/Dropbox/code/MLMC/code/SummaryStats.py:83: RuntimeWarning: invalid value encountered in multiply
  return self.m2/(self.count - 1) if (self.count > 1) else (math.inf * self.m2)


In [21]:
# set up MLMC coordinator with link to system
mcc = MCCoordinator.MCCoordinator(factory=system, 
                                  ml_hierarchy=['NoStore'], 
                                  use_expectations=False,
                                  use_joblib=False, joblib_n_jobs=-1, joblib_batch_size=5)

mcc.explore(n_samples=20)
for i in range(10):
    mcc.run_recommended(time_seconds=60, optimization_target='EENS')
    
mcc.verbose_result()

Running: {frozenset({'NoStore'}): 20}
relative sample numbers:  {frozenset({'NoStore'}): 49189.21804035297}
Running: {frozenset({'NoStore'}): 13186}
relative sample numbers:  {frozenset({'NoStore'}): 179679.72722426293}
Running: {frozenset({'NoStore'}): 16436}
relative sample numbers:  {frozenset({'NoStore'}): 177945.43521757075}
Running: {frozenset({'NoStore'}): 16196}
relative sample numbers:  {frozenset({'NoStore'}): 178224.82287144059}
Running: {frozenset({'NoStore'}): 16458}
relative sample numbers:  {frozenset({'NoStore'}): 179045.71544893793}
Running: {frozenset({'NoStore'}): 16801}
relative sample numbers:  {frozenset({'NoStore'}): 177524.10475070056}
Running: {frozenset({'NoStore'}): 16871}
relative sample numbers:  {frozenset({'NoStore'}): 177315.40694473992}
Running: {frozenset({'NoStore'}): 16795}
relative sample numbers:  {frozenset({'NoStore'}): 177633.98593589535}
Running: {frozenset({'NoStore'}): 16841}
relative sample numbers:  {frozenset({'NoStore'}): 178000.027158484

In [20]:
# set up MLMC coordinator with link to system
mcc = MCCoordinator.MCCoordinator(factory=system, 
                                  ml_hierarchy=['NoStore'], 
                                  use_expectations=True,
                                  use_joblib=False, joblib_n_jobs=-1, joblib_batch_size=5)

mcc.explore(n_samples=20)
for i in range(10):
    mcc.run_recommended(time_seconds=60, optimization_target='EENS')
    
mcc.verbose_result()

Running: {frozenset({'NoStore'}): 20}
relative sample numbers:  {frozenset({'NoStore'}): 1.0}
Running: {frozenset({'NoStore'}): 1}
relative sample numbers:  {frozenset({'NoStore'}): 1.0}
Running: {frozenset({'NoStore'}): 1}
relative sample numbers:  {frozenset({'NoStore'}): 1.0}
Running: {frozenset({'NoStore'}): 1}
relative sample numbers:  {frozenset({'NoStore'}): 1.0}
Running: {frozenset({'NoStore'}): 1}
relative sample numbers:  {frozenset({'NoStore'}): 1.0}
Running: {frozenset({'NoStore'}): 1}
relative sample numbers:  {frozenset({'NoStore'}): 1.0}
Running: {frozenset({'NoStore'}): 1}
relative sample numbers:  {frozenset({'NoStore'}): 1.0}
Running: {frozenset({'NoStore'}): 1}
relative sample numbers:  {frozenset({'NoStore'}): 1.0}
Running: {frozenset({'NoStore'}): 1}
relative sample numbers:  {frozenset({'NoStore'}): 1.0}
Running: {frozenset({'NoStore'}): 1}
relative sample numbers:  {frozenset({'NoStore'}): 1.0}
Running: {frozenset({'NoStore'}): 1}

Wall clock time: 0.764 s 	Total